In [1]:
import re
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver import ActionChains

# ElementClickInterceptedException - неможна клацнуть по елементу
# NoSuchElementException - такого елементу неіснує

In [293]:
def get_driver():
    path_to_driver = "./chromedriver"
    chrome_service = Service(path_to_driver)
    options = Options()
#     options.add_argument("headless")
    return webdriver.Chrome(
        service=chrome_service,
        options=options
    )


def find_elems(target, attr, attr_data):
    
    if attr == By.XPATH :
        attr_data =  f"//div[starts-with(@class, '{attr_data}')]"  
    
    count = 0
    while count < 6:
        try:
            elements = target.find_elements(attr, attr_data)
            if len(elements) == 0:
                time.sleep(0.5)
                count += 1
                continue
            break
        except NoSuchElementException:
            time.sleep(0.5)
            count += 1
            
    return elements


def click_elem(driver, element):
    """відкриття нового вікна з потрібним матчем"""
    
    act = ActionChains(driver)
    while 1:
        try:
            element.click()  
            break
        except Exception as e:
            act.key_down(Keys.DOWN).key_down(Keys.DOWN).perform()
            time.sleep(0.5)


def choose_day(driver, days, to_day='yesterday'):
    """
    вчорашні/завтрашні матчі ?! yesterday/tomorrow
    """
    for _ in range(days):
#         element = driver.find_element(By.CLASS_NAME, f"calendar__navigation--{to_day}")   
#         element.click()
        elements = find_elems(driver, By.CLASS_NAME, f"calendar__navigation--{to_day}")
        click_elem(driver, elements[0])
        
        
def get_source_w_ids(driver, days=1, to_day='yesterday'):
    driver.get("https://www.soccerstand.com/ru/")
    choose_day(driver, days=days, to_day=to_day)
    find_elems(driver, By.CLASS_NAME, 'sportName soccer')
    return driver.page_source

    
def get_ids(data):
    soup = BeautifulSoup(data, 'html.parser')
    
    divs_w_ids = (soup
         .find('div', class_=re.compile("sportName soccer"))
         .find_all('div', class_=re.compile("event__match"))
        )
    
    return list(map(
        lambda x: x.attrs["id"][4:],
        divs_w_ids
    ))

    
def teams_name(soup):
    return (
        (soup
                .find("div", class_=re.compile("duelParticipant__home"))
                .find("a", class_=re.compile("participant__participantName"))
                     ).text,
       (soup
            .find("div", class_=re.compile("duelParticipant__away"))
            .find("a", class_=re.compile("participant__participantName"))
                 ).text
           )

def teams_score(soup):
    return (soup
                .find("div", class_=re.compile("duelParticipant__score"))
                .find("div", class_=re.compile("detailScore__wrapper"))
                     ).text#.split("-")


def get_res_by_half(soup):
    """
    знаходження результатів кожного тайма
    повертає: список результатів, як строчки
    """
    f_half_raw, s_half_raw = soup.find_all('div', class_="smv__incidentsHeader section__title")
    return  f_half_raw.find_all('div')[1].text, s_half_raw.find_all('div')[1].text


def close_cookies_div(driver):
    """
    закриття вікна з підтвердження куків
    """
    global GLOBALS 
    
    elems = find_elems(driver, By.ID, 'onetrust-accept-btn-handler')
    if not elems:
        return
    elems[0].click()
    GLOBALS['close_cookies'] = True
    

def get_res_data(driver):
    elems = find_elems(driver, By.XPATH, "smv__incidentsHeader")        
    if not elems:
        raise Exception('нет данних за 1 тайм ')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    h_team, a_team = teams_name(soup)
    score = teams_score(soup)
    f_half, s_half = get_res_by_half(soup)
    return {
        "h_team": h_team,
        "a_team": a_team,
        "score": score,
        "f_half": f_half,
        "s_half": s_half,
    }


def do_job(driver, match_id):
    global GLOBALS 
    res_dict = {
        'current_match_data': None,
        'h_match_data': None,
        'a_match_data': None
    }
    
    
    driver.get(f"https://www.soccerstand.com/ru/match/{match_id}/#/match-summary/match-summary")

    if GLOBALS['close_cookies'] is False:
        close_cookies_div(driver)

    current_match_data = get_res_data(driver)

    res_dict['current_match_data'] = current_match_data

    driver.get(f"https://www.soccerstand.com/ru/match/{match_id}/#/standings/table/overall")

    team_rows = find_elems(driver, By.XPATH, 'ui-table__row table__row--selected' )

    for team in team_rows:
        line_data_row = team.text
        matches_elements = find_elems(team , By.CLASS_NAME, 'tableCellFormIcon' )


        target_match = 2 if '?' in line_data_row else 1

        click_elem(driver, matches_elements[target_match])

        # переключення на нове вікно
        w =  driver.window_handles    
        driver.switch_to.window(w[1])
        try:       
            window_data = get_res_data(driver)
            if current_match_data['h_team'] == line_data_row.split('\n')[1]:
                res_dict['h_match_data'] = window_data
            else:
                res_dict['a_match_data'] = window_data
        finally:
            driver.close()
            driver.switch_to.window(w[0])

    return res_dict 

In [ ]:
def main_job():
    try:
        rez_list = []
        driver = get_driver()
        
        sourse_w_ids = get_source_w_ids(driver)
        match_ids = get_ids(sourse_w_ids)
#         match_ids = ['UFdEv4Db', '8085tQrn']
        for match_id in match_ids[89:]:
            try:
                row_data = do_job(driver, match_id)
                if row_data is None:
                    continue
                rez_list.append(row_data)
#                 print(row_data['current_match_data']['h_team'])
            except Exception as e:
                print(e, type(e), match_id)
                continue
    finally:
        driver.quit()
    return rez_list
    
    
GLOBALS = {'close_cookies': False}   
rez = main_job()

not enough values to unpack (expected 2, got 1) <class 'ValueError'> UFdEv4Db
not enough values to unpack (expected 2, got 1) <class 'ValueError'> 8085tQrn
not enough values to unpack (expected 2, got 1) <class 'ValueError'> rJMmg3Si
not enough values to unpack (expected 2, got 1) <class 'ValueError'> xtMihNsb


'7.\nТолука U20\n11\n5\n2\n4\n18:11\n17\n?\nН\nП\nB\nB\nB'

In [273]:
headers = [
    'cur_match_h_team',
    'cur_match_a_team',
    'cur_match_score',
    'cur_match_f_half',
    'cur_match_s_half',
    
    'h_match_h_team',
    'h_match_a_team',
    'h_match_score',
    'h_match_f_half',
    'h_match_s_half',
    
    'a_match_h_team',
    'a_match_a_team',
    'a_match_score',
    'a_match_f_half',
    'a_match_s_half',
]

data = [
    a['current_match_data']['h_team'],
    a['current_match_data']['a_team'],
    a['current_match_data']['score'],
    a['current_match_data']['f_half'],
    a['current_match_data']['s_half'],
    
    a['h_match_data']['h_team'],
    a['h_match_data']['a_team'],
    a['h_match_data']['score'],
    a['h_match_data']['f_half'],
    a['h_match_data']['s_half'],
    
    a['a_match_data']['h_team'],
    a['a_match_data']['a_team'],
    a['a_match_data']['score'],
    a['a_match_data']['f_half'],
    a['a_match_data']['s_half'],
]
rez

[{'current_match_data': {'h_team': 'Астон Вилла',
   'a_team': 'Борнмут',
   'score': '3-0',
   'f_half': '1 - 0',
   's_half': '2 - 0'},
  'h_match_data': {'h_team': 'Вест Хэм',
   'a_team': 'Астон Вилла',
   'score': '1-1',
   'f_half': '1 - 1',
   's_half': '0 - 0'},
  'a_match_data': {'h_team': 'Борнмут',
   'a_team': 'Ливерпуль',
   'score': '1-0',
   'f_half': '1 - 0',
   's_half': '0 - 0'}},
 {'current_match_data': {'h_team': 'Брентфорд',
   'a_team': 'Лестер',
   'score': '1-1',
   'f_half': '1 - 0',
   's_half': '0 - 1'},
  'h_match_data': {'h_team': 'Саутгемптон',
   'a_team': 'Брентфорд',
   'score': '0-2',
   'f_half': '0 - 1',
   's_half': '0 - 1'},
  'a_match_data': {'h_team': 'Лестер',
   'a_team': 'Челси',
   'score': '1-3',
   'f_half': '1 - 2',
   's_half': '0 - 1'}},
 {'current_match_data': {'h_team': 'Вулверхэмптон',
   'a_team': 'Лидс',
   'score': '2-4',
   'f_half': '0 - 1',
   's_half': '2 - 3'},
  'h_match_data': {'h_team': 'Ньюкасл Юнайтед',
   'a_team': 'Вулв

In [266]:
def someFoo():
    global GLOBALS
    GLOBALS['close_cookies'] = True
    
GLOBALS = {'close_cookies': False}
 
someFoo()
print(GLOBALS)

{'close_cookies': True}
